# 하아르 크롤링 수행

- Iframe 구조로 된 페이지이므로 bs4, selenium 동시 사용
- 기본적인 크롤링 제한이 걸려있으므로, request 지속 수행시 ip벤의 위함이 존재함
    - proxy 사용 고려 (데이터 크롤링 속도가 더 느려질 수 있음)
    - 지속적인 header 값 변경 (header 데이터를 여러개 준비해야 함)

In [ ]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import random
import pandas as pd

In [ ]:
# 마지막 페이지 5108

l1 = [1, 1001, 2002, 3001,4001,5001]
l2 = [1001, 2001, 3001, 4001, 5109]

for k in range(len(l1)):
    data = []
    for j in range(l1[k],l2[k]):

        print("[", j, "실행", end='')

        # 크롬드라이버 실행
        driver = webdriver.Chrome()

        #크롬 드라이버에 url 주소 넣고 실행
        driver.get(f'https://thehar.co.kr/product/%ED%95%98%EC%95%84%EB%A5%B4-%EB%8B%A5%ED%84%B0%EB%AA%A8%EB%A6%AC%EC%97%94-100ml-drmori-n/9/category/28/display/1/#snapreview_widget3_0_page={j}')

        # 페이지가 완전히 로딩되도록 2+(0~2)초동안 기다림
        time.sleep(3+int((random.random())*10)%3)

        # iframe 전환
        driver.switch_to.frame('review_widget3_0')

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # exception 발생을 대비해 예외 처리 
        try:
        
            for i in range(1,6):
            
                name = ''
                score = 0
                text = ''
                recommend = 0
                selector = f"#snapreview_contents > div.sf_main_review > div.sf_review_detail_list > div:nth-child({i})"

                if soup.select(selector) != None:

                    # 이름
                    result_name = soup.select_one(selector + " > div.sf_review_block_right > div.sf_review_user_writer > div.sf_review_user_writer_name > span.value.font_montserrat") 
                    name = result_name.text

                    # 별점
                    result_score = soup.select(selector +" > div.sf_review_block_left > div.sf_review_user_top_area > div.sf_review_user_score > div > div")

                    if '100%' in str(result_score):
                        score = 5
                    elif '80%' in str(result_score):
                        score = 4
                    elif '60%' in str(result_score):
                        score = 3
                    elif '40%' in str(result_score):
                        score = 2
                    elif '20%' in str(result_score):
                        score = 1

                    # 본문
                    result_text = soup.select_one(selector +" > div.sf_review_block_left > div.sf_review_user_write.custom_font_size.custom_text_line > div.sf_review_user_write_review.sf_user_write_review.sf_text_overflow > span")

                    soup.find("div",{"class":"review_report_area"}).decompose()
                    text = result_text.text

                    # 추천
                    result_recommend = soup.select_one(selector + " > div.sf_review_block_left > div.sf_review_user_recommend > div.sf_review_user_recommend_score > div > div.sf_review_user_recommend_count > span.value" )
                    recommend = result_recommend.text

                data.append([name, int(score), text, int(recommend)])
                
        except:
                # exception 발생 시 해당 페이지 크롤링 스킵
            print(f"!!!{j}번째 수행 실패!!!")
                
        print("완료 ]",end=' ')
            
    # 임시 파일 생성(버퍼 비우기용)
    temp = pd.DataFrame(data, columns=['name','score','text','recommend'])
    temp.to_excel(f"har{k}.xlsx")
    time.sleep(100)

# 생성된 데이터 조합

In [ ]:
# 크롤링 데이터 임시저장본
a = pd.read_excel("har0.xlsx")
b = pd.read_excel("har1.xlsx")
c = pd.read_excel("har3.xlsx")
d = pd.read_excel("har4.xlsx")
e = pd.read_excel("har5.xlsx")
f = pd.read_excel("har6.xlsx")

i = pd.concat([a,b,c,d,e,f]).drop(['Unnamed: 0'], axis=1).reset_index(drop=True)

# 크롤링 데이터 최종본
i.to_excel("final.xlsx")